In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class traitement:

    def __init__(self):
        print("Bienvenu au TP2")
        state = False
        try:
            path = input("Veuillez entrer le chemin complet vers le data set en question:\n")
            self.df = pd.read_csv(path) #, sep = ";"
            self.a = pd.DataFrame()
        except FileNotFoundError:
            print("Fichier introuvable, vérifier bien le chemin que vous venez de fournir\n")
        except pd.errors.ParserError: # Pour pouvoir ouvrir les fichiers csv ayant le delimiteur qui est soit "," soit ";" sinon on ne l'ouvre pas le fichier
            print("Changement du mode d'ouverture en fonction du délimiteur")
            try:
                self.df = pd.read_csv(path, sep = ";")
            except pd.errors.ParserError:
                print("Importation du data Impossible")
        finally:
            print("Importation du Data réussit")
            
    def shoWholeData(self):
        print("df est de dimension: ", self.df.shape)
        return self.df.head()
#     def showData(self):
#         return self.a.head(10)

    def question1_1(self):
        nbre_actor = self.df.groupby("object_id").actor.unique().agg([len])
        nbre_session = self.df.groupby("object_id").session_uuid.agg([len])
        score_max_min = self.df.groupby("object_id").score.agg([max, min])
        mydf = pd.concat([nbre_actor, nbre_session, score_max_min], axis=1)
#         # mydf
#         type(mydf)
        mydf.columns.values[0] = "nbre_actor"
        mydf.columns.values[1] = "nbre_session"
        mydf = mydf.fillna(0)
        return mydf
    
    def question1_2(self):
        i_succes = self.df.loc[self.df["score"]>=50].groupby("object_id").actor.nunique()
        # i_succes
        i_echec = self.df.loc[self.df["score"]<50].groupby("object_id").actor.unique()
        # i_echec
        # df.loc[df["object_id"] == "632-le-hibou-grand-duc"]
        mydf = traitement.question1_1(self)
        mydf["succes_rate%"] = (i_succes/self.df.groupby('object_id').actor.nunique())*100
        mydf.sort_values(['succes_rate%'],ascending=[False],inplace=True)
        mydf = mydf.fillna(0)
        top10 = mydf.head(10)
        tab2D = np.array([top10])
        return mydf, tab2D
    
    def question2(self):
        top_des_modules = mydf.head(1)
        last_des_modules = mydf.tail(1)

        dataFrame_top_module = self.df.loc[self.df['object_id'] == top_des_modules.index[0]].groupby('actor').agg({"session_uuid":['count'],"temps":['sum'],"score":['max']})
        dataFrame_top_module['resultat'] = 0 
        dataFrame_top_module.loc[dataFrame_top_module.score['max'] >= 50, 'resultat'] = 1
        temps_total = dataFrame_top_module.temps/60
        dataFrame_top_module = pd.concat([dataFrame_top_module.session_uuid,dataFrame_top_module.temps,dataFrame_top_module.score,dataFrame_top_module.resultat],axis=1)
        dataFrame_top_module.rename({"count":"nb_sessions"}, axis=1,inplace=True)
        dataFrame_top_module.rename({"sum":"temps_total"}, axis=1,inplace=True)
        dataFrame_top_module.rename({"max":"score_max"}, axis=1,inplace=True)
        # ----------------------------------------------------------------------------------------------------------------------
        dataFrame_last_des_modules = self.df.loc[self.df['object_id'] == last_des_modules.index[0]].groupby('actor').agg({"session_uuid":['count'],"temps":['sum'],"score":['max']})
        dataFrame_last_des_modules['resultat'] = 0 
        dataFrame_last_des_modules.loc[dataFrame_last_des_modules.score['max'] >= 50, 'resultat'] = 1
        dataFrame_last_des_modules = pd.concat([dataFrame_last_des_modules.session_uuid,dataFrame_last_des_modules.temps,dataFrame_last_des_modules.score,dataFrame_last_des_modules.resultat],axis=1)
        dataFrame_last_des_modules.rename({"count":"nb_sessions"}, axis=1,inplace=True)
        dataFrame_last_des_modules.rename({"sum":"temps_total"}, axis=1,inplace=True)
        dataFrame_last_des_modules.rename({"max":"score_moyenne"}, axis=1,inplace=True)

        return dataFrame_top_module, dataFrame_last_des_modules

    def moduleGenerator(self, num, sorted_df, sorted_col):
        sorted_df = sorted_df.reset_index()
        #print(type(sorted_df))
        data = self.df.loc[self.df['object_id'] == sorted_df.loc[num, "object_id"]].groupby(["object_id", "actor"]).agg({"temps":['sum'],"score":['max']})
        data = pd.concat([data.temps, data.score],axis=1)
        data.rename({"count":"nb_sessions"}, axis=1,inplace=True)
        data.rename({"sum":"temps_total"}, axis=1,inplace=True)
        data.rename({"max":"score_max"}, axis=1,inplace=True)
        data = data.sort_values(by= sorted_col, ascending= False )
        return data

    def boucle(self, n, sorted_df, sorted_col):
        top = list()
        for i in range(n):
            top.append(traitement.moduleGenerator(self, i, sorted_df, sorted_col))
        return top

    def courbe(self, module, n):
        plt.figure(figsize=(10, 7))
        for i in range(n):
            # Courbe
            t_minute = module[i]["temps_total"]/60
            x= np.array(t_minute)
            y= np.array(module[i]["score_max"])
    #         print(x.shape)
    #         print(y.shape)
            plt.plot(x, y, label="Top"+ str(i)) 
            plt.xlabel('Temps(min)')
            plt.ylabel('Score_Max')
            plt.title("Simple Plot")
            plt.legend();

    def courbe2(self, module):
        rgActor = []
        scoreActor =[]
        fig, axs = plt.subplots(len(module), 1, figsize=(10, 10))
        for mod in range(len(module)):
            rgActor = np.arange(len(module[mod]))
            scor = module[mod]["score_max"]
    #         print(rgActor.shape)
    #         print(scor.to_numpy().shape)
            axs[mod].plot(rgActor, scor.to_numpy())


    # Diagramme en baton        
    def bar(self, module, n, sorted_df):
        plt.figure(figsize=(10, 7))
        sorted_df, mydf1 = traitement.question1_2(self)
#         return sorted_df.loc[0, 'succes_rate%']
        for i in range(n):
            module[i] = module[i].reset_index()
            module[i]["succes_rate"] = sorted_df.iloc[i, 4]
            plt.bar(module[i].object_id, module[i]["succes_rate"],  color = "#A0AAE4", edgecolor="red", linewidth=4, ecolor = "green",capsize = 10)  # Plot some data on the (implicit) axes.
            plt.xticks(rotation= "vertical")
            plt.title("Simple Plot")
            plt.ylim((90, 100))
        plt.show()

    def joinTimeToMydf(self, mydf):
        dftime = self.df.groupby("object_id").temps.agg(sum)
        dftime = pd.DataFrame(dftime)
#         mydf = mydf.set_index("object_id")
        mydf= mydf.sort_values("object_id")
        dftime= dftime.sort_index()
        t = mydf.join(dftime)
        t = t.sort_values(['succes_rate%'], ascending=False)
        return t


    def scatter(self, module):
        rgMod = range(len(module))
        mydf, inutil_ici = traitement.question1_2(self)
        mydf = traitement.joinTimeToMydf(self, mydf).head(len(module))
        plt.figure(figsize=(10, 7))
        plt.scatter(rgMod, (mydf["temps"]/60).to_numpy())
        plt.xlabel("Le Rang")
        plt.ylabel("Temps en min")


    def switch(self, data):
        if (0<=data<50):
            return "D"
        elif (50<=data<60):
            return "C"
        elif (60<=data<80):
            return "B"
        elif (80<=data<=100):
            return "A"
        else:
            return "Taux anormal"


    def cammenbert(self, sorted_df):
        liste1 = []
        a = sorted_df["succes_rate%"]
        liste1.append(a.map(lambda x:traitement.switch(self, x)))
        a = pd.DataFrame(a)
        liste2 = pd.DataFrame(liste1)     
        liste2 = (liste2.T)
        liste2.rename(columns = {"succes_rate%" : "Classe"}, inplace=True)
        dfinal = pd.concat([a, liste2], axis =1)
        dfinal = dfinal.groupby(["Classe"]).count()

        plt.figure(figsize = (8, 8))
        plt.pie(dfinal["succes_rate%"].values, labels = dfinal.index, 
               colors = ['red', 'green', 'yellow', 'blue'],
               explode = [0, 0.1, 0, 0],
               autopct = lambda x: str(round(x, 2)) + '%',
               pctdistance = 0.7, labeldistance = 1.1,
               shadow = False)
        plt.legend()

In [107]:
a = traitement()
# mydf, tab2D = a.question1_2()

a.shoWholeData()

#---------------Premième partie----------------------
# a.question1_1()

# tab2D

# a.question2()

#---------------Deuxième partie----------------------
#Les 3 lignes suivantes doivent toutes être decommentées avant l'execution de n'importe quelle question de cette 2ème partie
# n = int(input("Sur les Combien 1er modules voulez-vous travailler?\n"))
# m1= a.boucle(n, mydf, "temps_total")
# m2= a.boucle(n, mydf, "score_max")

# Question1
# a.courbe(m1, n)
# a.bar(m1, n, mydf)
# Question2
# a.courbe2(m2)
# Question3
# a.scatter(m1) # il s'agit de tous les 152 modules
# a.cammenbert(mydf)

Bienvenu au TP2
Veuillez entrer le chemin complet vers le data set en question:
C:/Users/Oussou/Documents/Rachid Ousmane/cours/S6/Intelligence Artificielle/automobileEDA.csv
Importation du Data réussit
df est de dimension:  (201, 29)


,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


In [ ]:
C:/Users/Oussou/Documents/Rachid Ousmane/cours auto/ODC_AI/Tp2/scorm.csv
    
C:\Users\Oussou\Documents\Rachid Ousmane\cours\S6\Intelligence Artificielle\automobileEDA.csv